In [3]:
import requests
from datetime import datetime, timedelta
import ccxt
import pandas as pd
import numpy as np
import talib
import settings
import time
import joblib
from logging import getLogger,Formatter,StreamHandler,FileHandler,INFO

# モデルの読み込み
model_y_buy = joblib.load('./model_y_buy.xz')
model_y_sell = joblib.load('./model_y_sell.xz')

# ccxtのパラメータ
symbol = 'BTC/JPY'	  # 購入予定のシンボル
product_code = 'FX_BTC_JPY'
bitflyer = ccxt.bitflyer()		 # 使用する取引所を記入
bitflyer.apiKey = settings.apiKey
bitflyer.secret = settings.secret


In [8]:
position = bitflyer.private_get_getpositions( params = { "product_code" : "FX_BTC_JPY" })
orders = bitflyer.fetch_open_orders(
	symbol = "FX_BTC_JPY",
	params = { "product_code" : "FX_BTC_JPY" })
print(position)
print(orders)

[]
[{'id': 'JRF20211231-040717-137786', 'clientOrderId': None, 'info': {'id': '0', 'child_order_id': 'JFX20211231-040717-173690F', 'product_code': 'FX_BTC_JPY', 'side': 'BUY', 'child_order_type': 'LIMIT', 'price': '5582151.0', 'average_price': '0.0', 'size': '0.01', 'child_order_state': 'ACTIVE', 'expire_date': '2022-01-30T04:07:17', 'child_order_date': '2021-12-31T04:07:17', 'child_order_acceptance_id': 'JRF20211231-040717-137786', 'outstanding_size': '0.01', 'cancel_size': '0.0', 'executed_size': '0.0', 'total_commission': '0.0'}, 'timestamp': 1640923637000, 'datetime': '2021-12-31T04:07:17.000Z', 'lastTradeTimestamp': None, 'status': 'open', 'symbol': 'FX_BTC_JPY', 'type': 'limit', 'timeInForce': None, 'postOnly': None, 'side': 'buy', 'price': 5582151.0, 'stopPrice': None, 'cost': 0.0, 'amount': 0.01, 'filled': 0.0, 'remaining': 0.01, 'fee': {'cost': 0.0, 'currency': None, 'rate': None}, 'average': None, 'trades': [], 'fees': [{'cost': 0.0, 'currency': None, 'rate': None}]}]
